# Graph RAG using Amazon Bedrock Knowledge Bases
In this module, you'll learn how to improve the Foundation Model (FM) generations by controlling the maximum no. of results retrieved and performing custom prompting in Knowledge bases (KB) for Amazon Bedrock.
This module contains:
1. [Overview](#1-Overview)
2. [Pre-requisites](#2-Pre-requisites)
3. [Understanding RetrieveAndGenerate API](#understanding-retrieveandgenerate-api)
4. [Sreaming response using RetrieveAndGenerate API](#streaming-response-with-retrieveandgenerate-api)
5. [Adjust 'maximum number of results' retrieval parameter](#3-how-to-leverage-the-maximum-number-of-results-feature)
6. [How to use custom prompting](#4-how-to-use-the-custom-prompting-feature)

## Overview
Amazon Bedrock Knowledge Bases offers a fully managed GraphRAG feature with Amazon Neptune. This functionality uses Retrieval Augmented Generation (RAG) techniques combined with graphs to enhance generative AI applications so that end users can get more accurate and comprehensive responses.

GraphRAG automatically identifies and uses relationships between related entities and structural elements (such as section titles) across documents that are ingested into Amazon Bedrock Knowledge Bases. This means that generative AI applications can deliver more relevant responses in cases where connecting data and reasoning across multiple document chunks is needed.

Amazon Bedrock Knowledge Bases automatically manages the creation and maintenance of the graphs from Amazon Neptune, so you can provide relevant responses to your end users, without relying on expertise in graph techniques.

Amazon Bedrock Knowledge Bases with GraphRAG offers the following benefits:

- More relevant responses by using contextual information from related entities and document sections.

- Better summarization by incorporating key content from your data sources while filtering out unnecessary information.

- More explainable responses by understanding the relationships between different entities in the dataset and providing citations.

**Note**: GraphRAG is available in AWS Regions where both Amazon Bedrock Knowledge Bases and Amazon Neptune Analytics are both available.

- For this module, we will use the Anthropic Claude 3 Haiku model as our FM to work with the max no. of results and prompt customization features

## Pre-requisites
Before being able to answer the questions, the documents must be processed and stored in a knowledge base. For this notebook, we use a `synthetic dataset for 10K financial reports` to create the Amazon Bedrock Knowledge Bases. 

1. Upload your documents (data source) to Amazon S3 bucket.

## Setup

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.2 requires botocore<1.35.37,>=1.35.16, but you have botocore 1.37.9 which is incompatible.
langchain-experimental 0.0.58 requires langchain<0.2.0,>=0.1.17, but you have langchain 0.3.20 which is incompatible.
langchain-experimental 0.0.58 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.43 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Initialize boto3 client
Through out the notebook, we are going to utilise RetrieveAndGenerate to test knowledge base features.

In [3]:
import json
import boto3
import pprint
import sys
from botocore.exceptions import ClientError
from botocore.client import Config
import time
import os
import logging

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
    
from utils.knowledge_base import BedrockKnowledgeBase

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()['Account']

# Create s3 client
s3_client = boto3.client('s3')

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-haiku-20240307-v1:0" # we will be using Anthropic Claude 3 Haiku throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'


In [4]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Graph RAG knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'
intermediate_bucket_name = f'{knowledge_base_name}-intermediate-{account_id}'

## Create KB with Amazon Neptune as vector
To build GraphRAG, you must choose Amazon Neptune Analytics as your vector store.
The knowledge base automatically generates and stores document embeddings in Amazon Neptune, along with a graph representation of entities and their relationships derived from the document corpus.

When your GraphRAG-based application is running, you can continue using the Knowledge Bases API operations to provide end users with more comprehensive, relevant, and explainable responses.


In [5]:
data_sources = [{"type": "S3", "bucket_name": bucket_name}]

In [6]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "GRAPH", 
    suffix = f'{suffix}-f',
    vector_store="NEPTUNE_ANALYTICS" # can be OPENSEARCH_SERVERLESS or NEPTUNE_ANALYTICS
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['bedrock-sample-knowledge-base-9192224-017444429555']
buckets_to_check:  ['bedrock-sample-knowledge-base-9192224-017444429555']
Creating bucket bedrock-sample-knowledge-base-9192224-017444429555
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9192224-f) and Policies
Step 3 - Creating Neptune Analytics Graph Index: might take upto 5-7 minutes
Graph is getting creating...
Graph is getting creating...
Graph is getting creating...
Graph is getting creating...
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is GRAPH
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 3, 10, 2, 29, 11, 864728, tzinfo=tzutc()),
  'description': 'Graph RAG knowledge base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:017444429555:knowledge-base/KEWHZ71GDS',
  'knowledgeBaseConfiguration': { 

ParamValidationError: Parameter validation failed:
Unknown parameter in vectorIngestionConfiguration: "graphContextConfiguration", must be one of: chunkingConfiguration, contextEnrichmentConfiguration, customTransformationConfiguration, parsingConfiguration

## Download data to ingest into our knowledge base.
We'll use the following data:

sythetic data stored in a local directory as first data source
### Upload data to S3 Bucket data source

In [ ]:
import os
s3_client = boto3.client('s3')

def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../synthetic_dataset", bucket_name)

uploading file ../synthetic_dataset/octank_financial_10K.pdf to bedrock-sample-knowledge-base-4165155-706553727873


## Start ingestion job
Once the KB and data source(s) created, we can start the ingestion job for each data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

NOTE: Currently, you can only kick-off one ingestion job at one time.

In [ ]:
# ensure that the kb is available
time.sleep(60)
# sync knowledge base
knowledge_base.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'LNM420II0C',
  'ingestionJobId': 'SHFWT4WMZ4',
  'knowledgeBaseId': 'D74BUZ1HYI',
  'startedAt': datetime.datetime(2025, 3, 4, 17, 0, 20, 929626, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 3, 4, 17, 1, 5, 578107, tzinfo=tzlocal())}


In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

'D74BUZ1HYI'
Stored 'kb_id' (str)


## 2.2 Test the Knowledge Graph created using Amazon Bedrock Knowledge Bases
You can use the same `retreive` or `retrieve_and_generate` API based on your use case to query the `Neptune Analytics` graph created using Amazon Bedrock Knowledge Bases.

### Testing Knowledge Base with Retrieve and Generate API
We will first test it with `retrieve_and_generate` API which will now query the `Neptune Analytics` graph created using Amazon Bedrock Knowledge Bases to fetch the search results, augment it with the prompt and then use the provided foundation model to generate the response. 

`query = Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

The right response for this query as per ground truth QA pair is:

```
According to the consolidated statements of cash flows, in the fiscal year ended December 31, 2019, Octank Financial had:
- Net cash provided by operating activities of $710 million
- Net cash used in investing activities of $240 million
- Net cash provided by financing activities of $350 million
- An overall net increase in cash and cash equivalents of $120 million
```

**NOTE**: Based on the model selected for generating responses, actual response may differ, however, will be semantically similar. 

In [ ]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019?"


In [ ]:
# foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)


According to the consolidated statements of cash flows, in the fiscal year ended December 31, 2019, Octank Financial had:
- Net cash provided by operating activities of $710 million
- Net cash used in investing activities of $240 million
- Net cash provided by financing activities of $350 million
- An overall net increase in cash and cash equivalents of $120 million



As you can see, with the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chunks that best match your query using the retrieve API. Since, the vector store is `Neptune Analytics`, the retrieve API will query the graph to fetch the relevant chunks (search results). In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [ ]:
response_ret = bedrock_agent_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)


Chunk 1:  In conclusion, Octank Financial's cash and cash equivalents have increased significantly in 2021 due to strategic cash management and short-term investment decisions.

Chunk 1 Location:  {'s3Location': {'uri': 's3://bedrock-sample-knowledge-base-4165155-706553727873/octank_financial_10K.pdf'}, 'type': 'S3'}

Chunk 1 Score:  1.74458

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://bedrock-sample-knowledge-base-4165155-706553727873/octank_financial_10K.pdf', 'x-amz-bedrock-kb-document-page-number': 110.0, 'x-amz-bedrock-kb-chunk-id': '1b1a33d0-9fac-4bde-9da9-8c51abf57875', 'x-amz-bedrock-kb-data-source-id': 'LNM420II0C'}

Chunk 2:  This increase was primarily due to higher marketing and advertising expenses, as well as increased research and development costs associated with the company's growth.     Operating income for 2021 was $1,800 million, an increase of $100 million or 6% compared to $1,700 million in 2020. This growth was driven by the increase in gross profit,

## Clean up
Please make sure to uncomment and run the below section to delete all the resources.

In [13]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Deleting Knowledge Base and associated resources==============================

Deleted data source LNM420II0C
======== Knowledge base and all data sources deleted =========
Found bucket bedrock-sample-knowledge-base-4165155-706553727873
Deleted all objects in bucket bedrock-sample-knowledge-base-4165155-706553727873
Deleted bucket bedrock-sample-knowledge-base-4165155-706553727873
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_4165155-f
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_4165155-f========
 [{'PolicyName': 'AmazonBedrockNeptunePolicyForKnowledgeBase_4165155-f', 'PolicyArn': 'arn:aws:iam::706553727873:policy/AmazonBedrockNeptunePolicyForKnowledgeBase_4165155-f'}, {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_4165155-f', 'PolicyArn': 'arn:aws:iam::706553727873:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_4165155-f'}, {'P

<div class="alert alert-block alert-warning">
<b>Note:</b> Remember to delete KB, OSS index and related IAM roles and policies to avoid incurring any charges.
</div>